# RW - LM spec migration management v2 -> v4.

The aim of this script is to programatically allow us to move from the v2 layer spec from layer manager used in RW layers to the v4.

We will take as started point rw past migration and latest migration done by gfw. plus scripts develop for OW migration management from staging and productions


Tasks prior running the process:

* RW management functions
* Develop functions/code to move from spect v2 to v4
* Develop test to ensure functions works as expected

Finally the workflow will be excecuted one provider type at a time, and will be as follows:  

* Migrate to staging 5 layers of each provider
* For those 5 layers we will run the script to migrate and validate.
* Once Front is happy with the results we will trigger the full migration

In [54]:
### Run functions developed in the notebooks
%run './pydantic_data_classes_rw.ipynb'

In [55]:
%run './RW-API_management_functions.ipynb'

builtins==None
builtins==None
requests_toolbelt.sessions==None
types==None
json==2.0.9
logging==0.5.1.2
getpass==None
os==None
subprocess==None
ipykernel==6.4.1
debugpy==1.4.1
json==2.0.9
builtins==None


In [80]:
%run './RW-LM4_migration_functions.ipynb'


In [2]:
from devtools import debug

In [3]:
from IPython.display import JSON, display

In [7]:
session = rwApiSession()

You are login into RW api
Successfully logged into RW api


### Explore and get some metrics about the layers currently in RW

In [8]:
layerList = getLayerList(session)

In [9]:
display(f' currently available {len(layerList)} layers in the api')

' currently available 5437 layers in the api'

In [64]:
display(layerList[0])
JSON(layerList[0].json(exclude_none=True))

**Layer**: [December 12, 2021 00:00 CT O₃ Concentration (ppb)](https://api.resourcewatch.org/v1/dataset/00d6bae1-e105-4165-8230-ee73a8128538/layer/50168169-0ce3-41e7-8056-29d87a18e66d)

/opt/conda/lib/python3.8/site-packages/IPython/core/display.py:911: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [11]:
providers = {}
for layer in layerList:
    if layer.attributes.provider in providers.keys():
        providers[layer.attributes.provider]['count'] += 1
        providers[layer.attributes.provider]['ids'].append(layer.id)
    else:
        providers[layer.attributes.provider] = {'count': 1, 'ids':[layer.id]} 

for k,v in providers.items():
    print(f'{k}: {v["count"]}')

gee: 1594
cartodb: 3506
leaflet: 302
featureservice: 19
wms: 15
None: 1


This means in our migration we will need to only take care of 5 type of connectors

#### Review the layer with None as provider and exclude it from our migration

In [38]:
for layer in layerList:
    if layer.attributes.provider is None:
        display(layer)
        display(JSON(layer._url))

**Layer**: [Brazilian Amazon Deforestation Alerts](https://api.resourcewatch.org/v1/dataset/8870d62f-0240-4275-910a-361f0ef347e0/layer/0c07f80c-129d-47b3-9099-99036883a215)

<IPython.core.display.JSON object>

#### Lets have a look at the layerType that indicates a successfull migration from v1 to v2

In [39]:
layerTypes = {}
for layer in layerList:
    if layer.attributes.layerConfig.get('layerType') in layerTypes.keys():
        layerTypes[layer.attributes.layerConfig.get('layerType')] += 1
    else:
        layerTypes[layer.attributes.layerConfig.get('layerType')] = 1
layerTypes

{'vector': 2730, None: 1871, 'raster': 826}

#### From those that look unsusscessfull lets have a look at the providers, we will need to be extra carefull with this ones:

In [40]:
layer2Types = {}
for layer in layerList:
    if layer.attributes.layerConfig.get('layerType') is None:
        if layer.attributes.provider in layer2Types.keys():
            layer2Types[layer.attributes.provider] += 1
        else:
            layer2Types[layer.attributes.provider] = 1
layer2Types

{'gee': 1432,
 'cartodb': 282,
 'featureservice': 18,
 'wms': 15,
 'leaflet': 123,
 None: 1}

#### Add some testing layers from prod to staging from each different type so we can have some testing data.

In [14]:
testing_layers = []
try:
    for k,v in providers.items():
        print(f'{k}: ')
        for ID in v["ids"][:5]:
            layer = parse_obj_as(Layer, session.get(f'v1/layer/{ID}').json().get('data'))
            display(layer)
            copyLayer = layer.copy(deep=True)
            copyLayer.attributes.env = 'staging'
            testing_layers.append(session.post(f'v1/dataset/{copyLayer.attributes._datasetId}/layer', 
                                data = copyLayer.attributes.json(exclude_none=True))\
                                .json().get('data').get('id'))
except:
    pass

gee: 


**Layer**: [December 12, 2021 00:00 CT O₃ Concentration (ppb)](https://api.resourcewatch.org/v1/dataset/00d6bae1-e105-4165-8230-ee73a8128538/layer/50168169-0ce3-41e7-8056-29d87a18e66d)

**Layer**: [December 12, 2021 05:00 CT O₃ Concentration (ppb)](https://api.resourcewatch.org/v1/dataset/00d6bae1-e105-4165-8230-ee73a8128538/layer/ac3c0d6e-32c5-4044-9f6a-8b340794dae1)

**Layer**: [December 12, 2021 20:00 CT O₃ Concentration (ppb)](https://api.resourcewatch.org/v1/dataset/00d6bae1-e105-4165-8230-ee73a8128538/layer/15c0e0b6-0625-448f-8f8e-f0ec31726f92)

**Layer**: [December 11, 2021 08:00 CT O₃ Concentration (ppb)](https://api.resourcewatch.org/v1/dataset/00d6bae1-e105-4165-8230-ee73a8128538/layer/8a668d09-10ba-4bd9-977d-6ae78629410f)

...

**Layer**: [Projected Change (2041-2070) in Average Annual Precipitation Lower Emissions](https://api.resourcewatch.org/v1/dataset/ffe316c8-c059-4ed6-a5c9-bc640afe4d0e/layer/1e0931ef-39d3-4d30-ac56-923bf065ab45)

**Layer**: [Projected Average Annual Precipitation (2041_2070) Higher Emissions](https://api.resourcewatch.org/v1/dataset/90ff981d-200e-4010-be0e-b7b00768fa50/layer/4de30682-f19e-4d78-a568-5139ce38b376)

**Layer**: [Hurricane Tracks (Cat 3_5 Hurricanes Small Scale)](https://api.resourcewatch.org/v1/dataset/303bd52b-020f-4744-87c5-8da38fffd738/layer/a4c7856a-ac31-4b61-9e46-5be586bda386)

wms: 


**Layer**: [South America try 4](https://api.resourcewatch.org/v1/dataset/33156582-1d90-48d7-9db8-17330cae3afc/layer/db597954-9ec3-4b59-b60d-ac61e6e21ed2)

In [15]:
testing_layers

['9cd806aa-8a9f-490b-a039-373ba37867b9',
 '8049a6b8-d6db-4c20-a618-dc02cb1a3fc1',
 'db926aa3-8d41-4f51-bdf8-76f02503f516',
 '37f33e70-a3e0-4297-b11b-6f806df8a80e',
 'ee2b7885-b9de-4a79-9786-7092b04f9f0d',
 '36b7182f-0121-4315-90d6-3c5e7709f094',
 '7eff62f0-a556-4fdc-982c-c6f537259f25',
 'a5d6989f-7924-44c0-8425-5667161ecf2a',
 'eb8ba80d-546b-4bd9-8895-b666a21cc7d4',
 '4d60a15c-44a2-4c1f-b166-6e6f3b1bccb5',
 'f06d1d2a-6aac-4eef-b492-76cbccd28cb2',
 '45350970-50b1-420e-9e27-7849e86c9e31',
 '251ce297-b4eb-4e0e-aa1a-16118e52cdf8',
 '105229af-fa6c-4765-b394-db3ee38767e9',
 '4f1429cd-f175-42b9-87f7-ede9cfff7a36',
 'fe7e5ffb-6ed9-4702-9673-c30a421d0b6f',
 'c828cb09-fac6-417a-8bbd-e7e9aba78a52',
 '03f5b60c-d7af-47b6-bb78-507e261b5c37',
 '88098435-baec-4063-bc0a-5c57d8855b9d',
 'fae5a0cb-c138-4f10-8ee0-2e4e8e1b7b9e']

#### Start migration on the testing layers

In [82]:
createUpdatelayerSchemas(session, testing_layers)

  0%|          | 0/20 [00:00<?, ?it/s]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.09s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
  5%|▌         | 1/20 [00:03<01:05,  3.46s/it]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.08s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
 10%|█         | 2/20 [00:07<01:05,  3.61s/it]

...

warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
 95%|█████████▌| 19/20 [00:43<00:01,  1.98s/it]

yarn install v1.22.17
[1/4] Resolving packages...
success Already up-to-date.
Done in 0.08s.


warning Skipping preferred cache folder "/home/jovyan/.cache/yarn" because it is not writable.
warning Selected the next writable cache folder in the list, will be "/tmp/.yarn-cache-1000".
100%|██████████| 20/20 [00:45<00:00,  2.27s/it]

Done! 0 / 20  failed, 0 flagged.


{'flagged': [],
 'failed': [],
 'success': ['9cd806aa-8a9f-490b-a039-373ba37867b9',
  '8049a6b8-d6db-4c20-a618-dc02cb1a3fc1',
  'db926aa3-8d41-4f51-bdf8-76f02503f516',
  '37f33e70-a3e0-4297-b11b-6f806df8a80e',
  'ee2b7885-b9de-4a79-9786-7092b04f9f0d',
  '36b7182f-0121-4315-90d6-3c5e7709f094',
  '7eff62f0-a556-4fdc-982c-c6f537259f25',
  'a5d6989f-7924-44c0-8425-5667161ecf2a',
  'eb8ba80d-546b-4bd9-8895-b666a21cc7d4',
  '4d60a15c-44a2-4c1f-b166-6e6f3b1bccb5',
  'f06d1d2a-6aac-4eef-b492-76cbccd28cb2',
  '45350970-50b1-420e-9e27-7849e86c9e31',
  '251ce297-b4eb-4e0e-aa1a-16118e52cdf8',
  '105229af-fa6c-4765-b394-db3ee38767e9',
  '4f1429cd-f175-42b9-87f7-ede9cfff7a36',
  'fe7e5ffb-6ed9-4702-9673-c30a421d0b6f',
  'c828cb09-fac6-417a-8bbd-e7e9aba78a52',
  '03f5b60c-d7af-47b6-bb78-507e261b5c37',
  '88098435-baec-4063-bc0a-5c57d8855b9d',
  'fae5a0cb-c138-4f10-8ee0-2e4e8e1b7b9e']}

#### Once we are happy with the layer we will proceed to migrate all +5000 of them:

In [ ]:
for k,v in providers:
    print(k)
    createUpdatelayerSchemas(session, v["ids"])

### Aside:
##### encode/decode vector tiles for testing data purpouses

In [49]:
import mapbox_vector_tile
import requests

In [ ]:
mapbox_vector_tile.encode([
    {
        "name": "errorTileSquareLayer",
        "features": [{ "geometry":"POLYGON ((0 0, 0 4096, 4096 4096, 4096 0, 0 0))" }]
    },
    {
        "name": "errorTileStripesLayer",
        "features": [{ "geometry":"POLYGON ((0 0, 0 4096, 4096 4096, 4096 0, 0 0))" }]
    }
])

In [51]:
response = requests.get('<url>')
mapbox_vector_tile.decode(response.content)

{'layer0': {'extent': 4096,
  'version': 2,
  'features': [{'geometry': {'type': 'Polygon',
     'coordinates': [[[3978, 3191],
       [4063, 3320],
       [4232, 3320],
       [4316, 3191],
       [4232, 3061],
       [4063, 3061],
       [3978, 3191]]]},
    'properties': {'puGeomId': '337a2b08-3fdc-46ea-9580-2ac5ee93b5c8',
     'scenarioPuId': '6424c619-6ccf-454b-ae50-bccf0c6645de',
     'puid': 27,
     'parseKeys': 'valuePosition,featureList',
     'costValue': 1.0},
    'id': 0,
    'type': 3},
   {'geometry': {'type': 'Polygon',
     'coordinates': [[[3978, 2930],
       [4063, 3061],
       [4232, 3061],
       [4316, 2930],
       [4232, 2800],
       [4063, 2800],
       [3978, 2930]]]},
    'properties': {'puGeomId': '12a227a2-e017-4335-afae-cfba9d381442',
     'scenarioPuId': '6f962395-48aa-492c-9ef8-d971b86259e2',
     'puid': 29,
     'parseKeys': 'valuePosition,featureList',
     'costValue': 1.0},
    'id': 0,
    'type': 3},
   {'geometry': {'type': 'Polygon',
     'co